In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import time
import sys
import pandas as pd
import json

Model Definition

In [ ]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Change input size
        self.input = nn.Linear(573, 128)
        self.hidden1 = nn.Linear(128, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.hidden3 = nn.Linear(128, 128)
        self.output = nn.Linear(128, 1)
        
    def forward(self, x):
        x = F.relu(self.input(x))
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = self.output(x)
        return x


Variables Definintions

In [ ]:
t = int(time.time())
useCUDA = True
dataPath = "../large_field_preprocessed_data.csv"
epochs = 100
batchSize = 32
modelPath = f"../trained_models/{t}"

Device Check

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if not useCUDA:
	device = torch.device('cpu')

Load data

In [ ]:
# data = pd.read_csv(dataPath)

Additional Format

In [ ]:
# dropColumns = [
#     "SERIAL",
#     "PERNUM",
#     "HHWT",
#     "CLUSTER",
#     "STRATA",
#     "PERWT",
#     "YRMARR",
#     "YRNATUR",
#     "RACNUM",
#     "index",
    
# ]

# data = data.drop(columns=dropColumns)

# for column in data.columns:
# 	data[column].fillna(False, inplace=True)
 
# data = data.drop(data[(data['INCWAGE_CPIU_2010'] == 0)].index)


Data Typing

In [ ]:
# print(data.shape)
# print(data.columns)

# # set to float32
# data['INCWAGE_CPIU_2010'] = data['INCWAGE_CPIU_2010'].astype(np.float32)

# for column in data.columns:
# 	if column == "INCWAGE_CPIU_2010":
# 		continue
# 	data[column] = data[column].astype(np.float32)


Tensor Creation

In [ ]:
# xTensor = torch.tensor(data.drop(columns=['INCWAGE_CPIU_2010']).values, dtype=torch.float32)
# yTensor = torch.tensor(data['INCWAGE_CPIU_2010'].values, dtype=torch.float32)

Model and Dataset Creation

In [ ]:
net = Net()
net = net.to(device)

# xTensor = xTensor.to(device)
# yTensor = yTensor.to(device)

# print(device)

# # create dataset
# dataset = TensorDataset(xTensor, yTensor)

trainSet = torch.load("../trainSet.pt")
trainSet = trainSet.to(device)
# create dataloader for both train and test
trainLoader = DataLoader(trainSet, batch_size=batchSize, shuffle=True)

Training

In [ ]:

optimizer = optim.Adam(net.parameters(), lr =1e-5)

criterion = nn.MSELoss(reduction='mean')

# trainLoader = trainLoader.to(device)


print("Epochs Started")

for epoch in range(epochs):
	running_loss = 0.0
	for i, data in enumerate(trainLoader):
		X, y = data
		y = y.unsqueeze(1)
  
		net.zero_grad()

		output = net(X)

		loss = criterion(output, y)
		loss.backward()
		optimizer.step()
		running_loss += loss.item()
		if i % 1000 == 999:    # print every 1000 mini-batches
			print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 1000:.3f}')
			running_loss = 0.0

torch.save(net.state_dict(), modelPath)
torch.cuda.empty_cache()
